In [1]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
import numpy as np
import time
#!pip install qgrid
import qgrid
#!pip install catboost
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Con pre procesamiento

In [3]:
def loadCSV(pathSamples, pathMatrix):
    df_loaded = pd.read_table(pathMatrix, sep=',')
    data = pd.read_table(pathSamples, sep=',')
    clin_trial_values = df_loaded.values
    columns = df_loaded.columns
    
    Y = data['Eligible']
    Y = Y.astype(int)
    X = clin_trial_values[:, :]
    return X,Y, columns

In [4]:
X, Y, columns = loadCSV("/content/drive/My Drive/Archivos Clin-trial/Datos/10k_1Col_NoCarEsp_LSA.csv", "/content/drive/My Drive/Archivos Clin-trial/Datos/docsTopicsLSA1500.csv") #Cargar SCV

In [5]:
def catboostPre(X,Y, Iteraciones = 2,learning_rate=0.1,  l2_leaf_reg = 3, depth = 5, model_size_reg = 5, random_strength = 1):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2)
    time_finish = time.time()
    # cargamos las 4 combinaciones de las compuertas XOR
    
    training_data = Xtrain

    # y estos son los resultados que se obtienen, en el mismo orden
    target_data = Ytrain
    
    model = CatBoostClassifier(boosting_type = 'Plain', iterations= Iteraciones, learning_rate=learning_rate, 
                               l2_leaf_reg = l2_leaf_reg, depth=depth,model_size_reg = model_size_reg, 
                               random_strength = random_strength)
    model.fit(training_data, target_data, verbose=False)

    preds = model.predict(Xtest)
    pred_train = model.predict(Xtrain)
    
    accuracyTrain = accuracy_score(target_data.values, pred_train)
    
    accuracyTest = accuracy_score(Ytest.values, preds)

    return model, str(time.time() - time_finish), accuracyTrain, accuracyTest

In [7]:
modelo, time_finish, accuracyTrain, accuracyTest = catboostPre(X,Y,Iteraciones=2000,learning_rate=0.3,
                                                                l2_leaf_reg = 7, depth = 1, model_size_reg=5,
                                                                  random_strength = 0.5)
print('Accuracy train: ',accuracyTrain* 100.0,' Accuracy test: ', accuracyTest* 100.0,
      'Tiempo de ejecución: ', time_finish)

Accuracy train:  95.25  Accuracy test:  76.5 Tiempo de ejecución:  91.32081651687622


In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Iteraciones' : pd.Series([500,1000,2000,2500,3000]), 
    'learning_rate' : pd.Series([0.3,0.3,0.3,0.3,0.3]), 
    'l2_leaf_reg' : pd.Series([7,7,7,7,7]),
    'depth' : pd.Series([1,1,1,1,1]),
    'model_size_reg' : pd.Series([5,5,5,5,5]),
    'random_strength' : pd.Series([0.5,0.5,0.5,0.5,0.5])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Iteraciones','learning_rate',  'l2_leaf_reg', 'depth', 'model_size_reg', 'random_strength'], inplace=True)

for Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = catboostPre(X,Y,Iteraciones=Iteraciones,learning_rate=learning_rate,
                                                                l2_leaf_reg = l2_leaf_reg, depth = depth, model_size_reg=model_size_reg,
                                                                  random_strength = random_strength)   
    
    
    df_types["Accuracy test"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = time_finish
    df_types["Accuracy train"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

,,,,,,Accuracy train,Accuracy test,Tiempo de ejecución
Iteraciones,learning_rate,l2_leaf_reg,depth,model_size_reg,random_strength,,,
500,0.3,7,1,5,0.5,84.3875,75.5,8.420408487319946
1000,0.3,7,1,5,0.5,89.6625,77.2,12.796949625015259
2000,0.3,7,1,5,0.5,94.4375,75.5,20.43360185623169
2500,0.3,7,1,5,0.5,95.55,78.25,24.357035636901855
3000,0.3,7,1,5,0.5,96.775,76.2,29.650611400604248


In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Iteraciones' : pd.Series([1000,1000,1000,1000,1000]), 
    'learning_rate' : pd.Series([0.3,0.3,0.3,0.3,0.3]), 
    'l2_leaf_reg' : pd.Series([7,7,7,7,7]),
    'depth' : pd.Series([1,2,3,4,5]),
    'model_size_reg' : pd.Series([5,5,5,5,5]),
    'random_strength' : pd.Series([0.5,0.5,0.5,0.5,0.5])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Iteraciones','learning_rate',  'l2_leaf_reg', 'depth', 'model_size_reg', 'random_strength'], inplace=True)

for Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = catboostPre(X,Y,Iteraciones=Iteraciones,learning_rate=learning_rate,
                                                                l2_leaf_reg = l2_leaf_reg, depth = depth, model_size_reg=model_size_reg,
                                                                  random_strength = random_strength)   
    
    
    df_types["Accuracy test"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = time_finish
    df_types["Accuracy train"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

Accuracy train  \
Iteraciones learning_rate l2_leaf_reg depth model_size_reg random_strength                  
1000        0.3           7           1     5              0.5                       89.5   
                                      2     5              0.5                     99.375   
                                      3     5              0.5                    99.9875   
                                      4     5              0.5                        100   
                                      5     5              0.5                     99.975   

                                                                           Accuracy test  \
Iteraciones learning_rate l2_leaf_reg depth model_size_reg random_strength                 
1000        0.3           7           1     5              0.5                      74.6   
                                      2     5              0.5                      76.7   
                                      3     5              0.5                     76.55   
                                      4     5              0.5                      78.3   
                                      5     5              0.5                      77.7   

                                                                           Tiempo de ejecución  
Iteraciones learning_rate l2_leaf_reg depth model_size_reg random_strength                      
1000        0.3           7           1     5              0.5              15.656417608261108  
                                      2     5              0.5              18.799176931381226  
                                      3     5              0.5              23.139086961746216  
                                      4     5              0.5               27.08918571472168  
                                      5     5              0.5               32.98676252365112

In [ ]:
modelo, time_finish, accuracyTrain, accuracyTest = catboostPre(X,Y, Iteraciones = 300,learning_rate=0.3,  
                                                               l2_leaf_reg = 7, depth = 2, model_size_reg = 5)
print('Accuracy train: ',accuracyTrain* 100.0,' Accuracy test: ', accuracyTest* 100.0,
      'Tiempo de ejecución: ', time_finish)

Accuracy train:  90.525  Accuracy test:  76.3 Tiempo de ejecución:  9.11328673362732


In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Iteraciones' : pd.Series([300,400,500,600,700]), 
    'learning_rate' : pd.Series([0.3,0.3,0.3,0.3,0.3]), 
    'l2_leaf_reg' : pd.Series([7,7,7,7,7]),
    'depth' : pd.Series([2,2,2,2,2]),
    'model_size_reg' : pd.Series([5,5,5,5,5]),
    'random_strength' : pd.Series([0.5,0.5,0.5,0.5,0.5])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Iteraciones','learning_rate',  'l2_leaf_reg', 'depth', 'model_size_reg', 'random_strength'], inplace=True)

for Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = catboostPre(X,Y,Iteraciones=Iteraciones,learning_rate=learning_rate,
                                                                l2_leaf_reg = l2_leaf_reg, depth = depth, model_size_reg=model_size_reg,
                                                                  random_strength = random_strength)   
    
    
    df_types["Accuracy test"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = time_finish
    df_types["Accuracy train"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

,,,,,,Accuracy train,Accuracy test,Tiempo de ejecución
Iteraciones,learning_rate,l2_leaf_reg,depth,model_size_reg,random_strength,,,
300,0.3,7,2,5,0.5,90.5375,75.45,9.520402908325195
400,0.3,7,2,5,0.5,93.4,75.7,10.390193939208984
500,0.3,7,2,5,0.5,95.0125,75.85,11.409874677658081
600,0.3,7,2,5,0.5,96.5125,76.45,12.715588808059692
700,0.3,7,2,5,0.5,97.7,77.55,13.61330771446228


In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Iteraciones' : pd.Series([500,500,500,500,500]), 
    'learning_rate' : pd.Series([0.3,0.3,0.3,0.3,0.3]), 
    'l2_leaf_reg' : pd.Series([7,7,7,7,7]),
    'depth' : pd.Series([2,2,2,2,2]),
    'model_size_reg' : pd.Series([5,5,5,5,5]),
    'random_strength' : pd.Series([0.5,0.7,1.5,1.7,1.2])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Iteraciones','learning_rate',  'l2_leaf_reg', 'depth', 'model_size_reg', 'random_strength'], inplace=True)

for Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = catboostPre(X,Y,Iteraciones=Iteraciones,learning_rate=learning_rate,
                                                                l2_leaf_reg = l2_leaf_reg, depth = depth, model_size_reg=model_size_reg,
                                                                  random_strength = random_strength)   
    
    
    df_types["Accuracy test"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = time_finish
    df_types["Accuracy train"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg, random_strength] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

Accuracy train  \
Iteraciones learning_rate l2_leaf_reg depth model_size_reg random_strength                  
500         0.3           7           2     5              0.5                    94.9125   
                                                           0.7                       95.3   
                                                           1.5                    94.9125   
                                                           1.7                     95.325   
                                                           1.2                    95.0375   

                                                                           Accuracy test  \
Iteraciones learning_rate l2_leaf_reg depth model_size_reg random_strength                 
500         0.3           7           2     5              0.5                      77.2   
                                                           0.7                     75.55   
                                                           1.5                     76.55   
                                                           1.7                      75.5   
                                                           1.2                      77.8   

                                                                           Tiempo de ejecución  
Iteraciones learning_rate l2_leaf_reg depth model_size_reg random_strength                      
500         0.3           7           2     5              0.5              11.154779434204102  
                                                           0.7              11.123818159103394  
                                                           1.5                11.0044527053833  
                                                           1.7              11.105925798416138  
                                                           1.2              10.932896852493286

## Learning curve

In [6]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, Y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes, scoring='f1',
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
## Plot LC

fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title = "Learning Curves (Catboost)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = CatBoostClassifier(boosting_type = 'Plain', iterations= 2000, learning_rate=0.3, 
                               l2_leaf_reg = 7, depth=1,model_size_reg = 5, 
                               random_strength = 0.5)

plot_learning_curve(estimator, title, X, Y, axes=axes[:, 0], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)
# Another stimator
#title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
#cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
#estimator = SVC(gamma=0.001)
#plot_learning_curve(estimator, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01),
#                    cv=cv, n_jobs=4)

plt.show()

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: ignored

## Sin pre procesamiento

In [ ]:
data = pd.read_csv('../../Dataset/10k_1Col_NoCarEsp_LSA.csv', sep=',')
Y = data['Eligible']
X = data
X = X.drop(['Eligible'], axis = 1)

In [ ]:
def catboost(X,Y, Iteraciones = 2,learning_rate=0.1,  l2_leaf_reg = 3, depth = 5, model_size_reg = 5):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2)
    time_finish = time.time()
    # cargamos las 4 combinaciones de las compuertas XOR
    train_pool = Pool(Xtrain, Ytrain, cat_features=['Interventions','Diagnoses'])
    test_pool = Pool(Xtest, Ytest, cat_features=['Interventions','Diagnoses'])
    training_data = Xtrain

    # y estos son los resultados que se obtienen, en el mismo orden
    target_data = Ytrain
    
    model = CatBoostClassifier(boosting_type = 'Plain', iterations= Iteraciones, learning_rate=learning_rate, 
                               l2_leaf_reg = l2_leaf_reg, depth=depth,  task_type="GPU", devices='0:1',
                              model_size_reg = model_size_reg)
    model.fit(train_pool, verbose=False)

    preds = model.predict(test_pool)
    pred_train = model.predict(train_pool)
    
    accuracyTrain = accuracy_score(target_data.values, pred_train)
    
    accuracyTest = accuracy_score(Ytest.values, preds)

    return model, str(time.time() - time_finish), accuracyTrain, accuracyTest

In [ ]:
modelo, time_finish, accuracyTrain, accuracyTest = catboost(X,Y, Iteraciones = 1000,learning_rate=0.5,  l2_leaf_reg = 3, depth = 5)

In [ ]:
print('Accuracy train: ',accuracyTrain* 100.0,' Accuracy test: ', accuracyTest* 100.0,
      'Tiempo de ejecución: ', time_finish)

Accuracy train:  100.0  Accuracy test:  53.7 Tiempo de ejecución:  17.92128825187683


In [ ]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    'Iteraciones' : pd.Series([200,200,200,200,200]), 
    'learning_rate' : pd.Series([0.8,0.8,0.8,0.8,0.8]), 
    'l2_leaf_reg' : pd.Series([7,7,7,7,7]),
    'depth' : pd.Series([2,3,4,5,6]),
    'model_size_reg' : pd.Series([5,5,5,5,5])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Iteraciones','learning_rate',  'l2_leaf_reg', 'depth', 'model_size_reg'], inplace=True)

for Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = catboost(X,Y,Iteraciones=Iteraciones,learning_rate=learning_rate,
                                                                l2_leaf_reg = l2_leaf_reg, depth = depth, model_size_reg=model_size_reg)   
    
    
    df_types["Accuracy test"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg] = time_finish
    df_types["Accuracy train"][Iteraciones, learning_rate, l2_leaf_reg, depth, model_size_reg] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

Accuracy train  \
Iteraciones learning_rate l2_leaf_reg depth model_size_reg                  
200         0.8           7           2     5                      99.975   
                                      3     5                     91.3625   
                                      4     5                     99.9875   
                                      5     5                       56.25   
                                      6     5                     80.6625   

                                                           Accuracy test  \
Iteraciones learning_rate l2_leaf_reg depth model_size_reg                 
200         0.8           7           2     5                       53.5   
                                      3     5                       52.2   
                                      4     5                       55.3   
                                      5     5                       55.5   
                                      6     5                       52.6   

                                                           Tiempo de ejecución  
Iteraciones learning_rate l2_leaf_reg depth model_size_reg                      
200         0.8           7           2     5                 3.01113224029541  
                                      3     5                3.629359245300293  
                                      4     5                3.482926845550537  
                                      5     5               3.7808728218078613  
                                      6     5                3.754887580871582